<a href="https://colab.research.google.com/github/dlwogns/ML-DL/blob/main/basic%20implementation/RNN_really_long_char.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np

In [3]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

In [4]:
char_set = list(set(sentence))
char_dic = {w : i for i, w in enumerate(char_set)}

In [5]:
#hyper parameter
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10 # 임의의 숫자를 넣는다.
learning_rate = 0.1

In [6]:
dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i : i+sequence_length]
  y_str = sentence[i+1 : i+sequence_length + 1]
  # 문장을 분할하여
  # 1. if you wan -> f you want
  # 2. f you want -> you want t
  # 이런식으로 순서대로 예측하게 데이터셋을 구축한다.
  print(i, x_str, '->', y_str)
  x = [char_dic[c] for c in x_str]
  y = [char_dic[c] for c in y_str]

  dataX.append(x)
  dataY.append(y)

batch_size = len(dataX)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [7]:
X_one_hot = tf.one_hot(dataX, num_classes)
Y_one_hot = tf.one_hot(dataY, num_classes)

In [10]:
X_one_hot.shape
# 170개의 배치, 10개의 sequence, 25개의 one hot size

TensorShape([170, 10, 25])

In [18]:
tf.model = tf.keras.Sequential()

In [19]:
tf.model.add(tf.keras.layers.LSTM(units = num_classes, input_shape=(sequence_length, num_classes), return_sequences=True))
#tf.model.add(tf.keras.layers.LSTM(units=num_classes, return_sequences = True))
tf.model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(units=num_classes, activation='softmax')))

In [20]:
tf.model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 10, 25)            5100      
                                                                 
 time_distributed (TimeDistr  (None, 10, 25)           650       
 ibuted)                                                         
                                                                 
Total params: 5,750
Trainable params: 5,750
Non-trainable params: 0
_________________________________________________________________


In [21]:
tf.model.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(lr = 0.1), metrics=['accuracy'])

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [22]:
tf.model.fit(X_one_hot, Y_one_hot, epochs = 100)

Epoch 1/100
6/6 [==============================] - 2s 8ms/step - loss: 2.9545 - accuracy: 0.1588
Epoch 2/100
6/6 [==============================] - 0s 6ms/step - loss: 2.3622 - accuracy: 0.3141
Epoch 3/100
6/6 [==============================] - 0s 7ms/step - loss: 1.6031 - accuracy: 0.4888
Epoch 4/100
6/6 [==============================] - 0s 7ms/step - loss: 1.1036 - accuracy: 0.6600
Epoch 5/100
6/6 [==============================] - 0s 7ms/step - loss: 0.7635 - accuracy: 0.7782
Epoch 6/100
6/6 [==============================] - 0s 9ms/step - loss: 0.5638 - accuracy: 0.8288
Epoch 7/100
6/6 [==============================] - 0s 8ms/step - loss: 0.4651 - accuracy: 0.8518
Epoch 8/100
6/6 [==============================] - 0s 7ms/step - loss: 0.4052 - accuracy: 0.8641
Epoch 9/100
6/6 [==============================] - 0s 7ms/step - loss: 0.3583 - accuracy: 0.8694
Epoch 10/100
6/6 [==============================] - 0s 7ms/step - loss: 0.3393 - accuracy: 0.8765
Epoch 11/100
6/6 [===========

In [27]:
results = tf.model.predict(X_one_hot)
#results
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j == 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

6/6 [==============================] - 0s 4ms/step
t you want to build a ship, don't arum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.